# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-8409da3664-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Literary Movements and Divine Comedy explorative search") 

Consider the following exploratory information need:

> investigate the literary movements and the poets. In particular, compare some italian poets in terms of place of birth, number of literary works and education.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q40185`| The Divine Comedy  | node |





Also consider

```
wd:Q40185 ?p ?obj .
```

is the BGP to retrieve all **properties of The Divine Comedy**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:

1. Who wrote The Divine Comedy? (return the IRI and the label of the author)

2. Identify the BGP for poet

3. Identify the BGP for literary movement

4. Return the poets born in the same place of the author of The Divine Comedy who are born between 1300 and 1500 (1300 and 1500 included) (the result set must be a list of couples poets IRI and label).

5. Identify the BGP for literary work

6. Compare the author of The Divine Comedy with Giovanni Boccaccio. Return for these poets the number of literary work they wrote (the result set must be two triples -one for each author- author IRI, label and #literary work).

7. Consider italian poets, their place of birth (the city) and their university education. Return for each city the number of poets who born there and who was educated there (the result set must be a list of triples city IRI, label and #poets).

8. Return the italian poets who won the Nobel Prize in Literature (the result set must be poet IRI and label). 

## Task 1
Who wrote The Divine Comedy? (return the IRI and the label of the author)

### Get all data property of "The Divine Comedy" (wd:Q40185)

In [4]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q40185 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1273'), ('label', 'CANTIC ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1296'), ('label', 'Gran Enciclopèdia Catalana ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1417'), ('label', 'Encyclopædia Britannica Online ID')]
[('property', 'http://www.wikidata.org/prop/direct/P214'), ('label', 'VIAF ID')]
[('property', 'http://www.wikidata.org/prop/direct/P227'), ('label', 'GND ID')]
[('property', 'http://www.wikidata.org/prop/direct/P244'), ('label', 'Library of Congress authority ID')]
[('property', 'http://www.wikidata.org/prop/direct/P268'), ('label', 'Bibliothèque nationale de France ID')]
[('property', 'http://www.wikidata.org/prop/direct/P269'), ('label', 'IdRef ID')]
[('property', 'http://www.wikidata.org/prop/direct/P3219'), ('label', 'Encyclopædia Universalis ID')]
[('property', 'http://www.wikidata.org/prop/direct/P3417'), ('label', 'Quora topic ID')]
[('property', 'http://www.wikidata.org/prop/dir

### Get all data property of "The Divine Comedy" (wd:Q40185) that do not contain ID

In [6]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q40185 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .
   FILTER REGEX(?label, "^((?!ID).)*$") . 
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('property', 'http://www.wikidata.org/prop/direct/P571'), ('label', 'inception')]
[('property', 'http://www.wikidata.org/prop/direct/P6839'), ('label', 'TV Tropes identifier')]
[('property', 'http://www.wikidata.org/prop/direct/P8248'), ('label', 'Colon Classification')]
[('property', 'http://www.wikidata.org/prop/direct/P8359'), ('label', 'Dewey Decimal Classification (works and editions)')]
5


### Get the class of "The Divine Comedy" (wd:Q40185)

In [51]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q40185 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)


Results
[('class', 'http://www.wikidata.org/entity/Q47461344'), ('label', 'written work')]
1


### Get all object properties of "The Divine Comedy" (wd:Q40185) 

In [7]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q40185 ?property ?object .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P2551'), ('label', 'used metre')]
[('property', 'http://www.wikidata.org/prop/direct/P1343'), ('label', 'described by source')]
[('property', 'http://www.wikidata.org/prop/direct/P136'), ('label', 'genre')]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P1889'), ('label', 'different from')]
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('label', 'instance of')]
[('property', 'http://www.wikidata.org/prop/direct/P407'), ('label', 'language of work or name')]
[('property', 'http://www.wikidata.org/prop/direct/P50'), ('label', 'author')]
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('label', 'has part')]
[('property', 'http://www.wikidata.org/prop/direct/P674'), ('label', 'characters')]
[('property', 'http://www.wikidata.org/prop/direct/P747'), ('label', 'has edition or translation')]
[('property', 'http://www.wikidata.org/p

### Get all the classes connected to "The Divine Comedy" (wd:Q40185)

In [11]:
queryString = """
SELECT DISTINCT ?objectClass ?label
WHERE {   
   wd:Q40185 ?property ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q581450'), ('label', 'fallen angel')]
[('objectClass', 'http://www.wikidata.org/entity/Q20830276'), ('label', 'group of mythical characters')]
[('objectClass', 'http://www.wikidata.org/entity/Q20902363'), ('label', 'mythical human-animal hybrid')]
[('objectClass', 'http://www.wikidata.org/entity/Q2280977'), ('label', 'Hellhound')]
[('objectClass', 'http://www.wikidata.org/entity/Q83203'), ('label', 'centaur')]
[('objectClass', 'http://www.wikidata.org/entity/Q4349921'), ('label', 'first human')]
[('objectClass', 'http://www.wikidata.org/entity/Q213924'), ('label', 'codex')]
[('objectClass', 'http://www.wikidata.org/entity/Q7553'), ('label', 'translation')]
[('objectClass', 'http://www.wikidata.org/entity/Q1751626'), ('label', 'theatre festival')]
[('objectClass', 'http://www.wikidata.org/entity/Q16326794'), ('label', 'mythological serpent')]
[('objectClass', 'http://www.wikidata.org/entity/Q2572794'), ('label', 'forme fixe')]
[('

### Get all the couples (object property - list of the classes of the objects that are connected to "The Divine Comedy"  (wd:Q40185) through that object property)

In [53]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?objectClass ; separator = ",")  AS ?objectsClasses) (GROUP_CONCAT(?labelObjectClass ; separator = ",")  AS ?labelsObjectsClasses)
WHERE 
{   
   wd:Q40185 ?property ?object .  
   ?object wdt:P31 ?objectClass .
   
   # Labels
   ?property sc:name ?labelProperty .      
   ?objectClass sc:name ?labelObjectClass .
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('labelProperty', 'has part'), ('objectsClasses', 'http://www.wikidata.org/entity/Q47461344,http://www.wikidata.org/entity/Q47461344,http://www.wikidata.org/entity/Q47461344,http://www.wikidata.org/entity/Q7725634'), ('labelsObjectsClasses', 'written work,written work,written work,literary work')]
[('property', 'http://www.wikidata.org/prop/direct/P136'), ('labelProperty', 'genre'), ('objectsClasses', 'http://www.wikidata.org/entity/Q223393'), ('labelsObjectsClasses', 'literary genre')]
[('property', 'http://www.wikidata.org/prop/direct/P407'), ('labelProperty', 'language of work or name'), ('objectsClasses', 'http://www.wikidata.org/entity/Q1288568,http://www.wikidata.org/entity/Q34770,http://www.wikidata.org/entity/Q1208380,http://www.wikidata.org/entity/Q661936'), ('labelsObjectsClasses', 'modern language,language,dialect group,pro-drop language')]
[('property', 'http://www.wikidata.org/prop/direct/P1889'), ('labelPr

### Get all the couples (object property - list of the  objects that are connected to "The Divine Comedy"  (wd:Q40185) through that object property)

In [55]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?object ; separator = ",")  AS ?objects) (GROUP_CONCAT(?labelObject; separator = ",")  AS ?labelsObjects)
WHERE 
{   
   wd:Q40185 ?property ?object .  
 
   # Labels
   ?property sc:name ?labelProperty .      
   ?object sc:name ?labelObject
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('labelProperty', 'has part'), ('objects', 'http://www.wikidata.org/entity/Q2713307,http://www.wikidata.org/entity/Q4509219,http://www.wikidata.org/entity/Q605018'), ('labelsObjects', 'Paradiso,Inferno,Purgatorio')]
[('property', 'http://www.wikidata.org/prop/direct/P136'), ('labelProperty', 'genre'), ('objects', 'http://www.wikidata.org/entity/Q37484'), ('labelsObjects', 'epic poem')]
[('property', 'http://www.wikidata.org/prop/direct/P407'), ('labelProperty', 'language of work or name'), ('objects', 'http://www.wikidata.org/entity/Q652,http://www.wikidata.org/entity/Q1990745'), ('labelsObjects', 'Italian,Tuscan')]
[('property', 'http://www.wikidata.org/prop/direct/P50'), ('labelProperty', 'author'), ('objects', 'http://www.wikidata.org/entity/Q1067'), ('labelsObjects', 'Dante Alighieri')]
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('labelProperty', 'instance of'), ('objects', 'http://www.wikidata.org/en

### Final Query

Final query for this task

In [54]:
queryString = """
SELECT DISTINCT ?author ?label
WHERE 
{   
   wd:Q40185 wdt:P50 ?author .
   ?author sc:name ?label .
}
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('author', 'http://www.wikidata.org/entity/Q1067'), ('label', 'Dante Alighieri')]
1


## Task 2
Identify the BGP for poet

### Get all data property of "Dante Alighieri" (wd:Q1067)

In [23]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q1067 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P7992'), ('label', "Poeti d'Italia in lingua latina author ID")]
[('property', 'http://www.wikidata.org/prop/direct/P1005'), ('label', 'Portuguese National Library ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1006'), ('label', 'Nationale Thesaurus voor Auteurs ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1015'), ('label', 'NORAF ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1017'), ('label', 'Vatican Library ID (former scheme)')]
[('property', 'http://www.wikidata.org/prop/direct/P1036'), ('label', 'Dewey Decimal Classification')]
[('property', 'http://www.wikidata.org/prop/direct/P1150'), ('label', 'Regensburg Classification')]
[('property', 'http://www.wikidata.org/prop/direct/P1207'), ('label', 'NUKAT ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1233'), ('label', 'Internet Speculative Fiction Database author ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1256'), (

### Get all data property of "Dante Alighieri" (wd:Q1067) that do not contain ID

In [25]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q1067 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .
   FILTER REGEX(?label, "^((?!ID).)*$") . 
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1036'), ('label', 'Dewey Decimal Classification')]
[('property', 'http://www.wikidata.org/prop/direct/P1150'), ('label', 'Regensburg Classification')]
[('property', 'http://www.wikidata.org/prop/direct/P1256'), ('label', 'Iconclass notation')]
[('property', 'http://www.wikidata.org/prop/direct/P1472'), ('label', 'Commons Creator page')]
[('property', 'http://www.wikidata.org/prop/direct/P2031'), ('label', 'work period (start)')]
[('property', 'http://www.wikidata.org/prop/direct/P213'), ('label', 'ISNI')]
[('property', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('property', 'http://www.wikidata.org/prop/direct/P4808'), ('label', 'Royal Academy new identifier')]
[('property', 'http://www.wikidata.org/prop/direct/P5587'), ('label', 'Libris-URI')]
[('property', 'http://www.wikidata.org/prop/direct/P569'), ('label', 'date of birth')]
[('property', 'http://www.wikidata.org/prop/direct/P570'), ('lab

### Get the class of "Dante Alighieri" (wd:Q1067)

In [26]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q1067 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q5'), ('label', 'human')]
1


### Get all object properties of "Dante Alighieri" (wd:Q1067)

In [27]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q1067 ?property ?object .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P101'), ('label', 'field of work')]
[('property', 'http://www.wikidata.org/prop/direct/P103'), ('label', 'native language')]
[('property', 'http://www.wikidata.org/prop/direct/P106'), ('label', 'occupation')]
[('property', 'http://www.wikidata.org/prop/direct/P1066'), ('label', 'student of')]
[('property', 'http://www.wikidata.org/prop/direct/P119'), ('label', 'place of burial')]
[('property', 'http://www.wikidata.org/prop/direct/P1196'), ('label', 'manner of death')]
[('property', 'http://www.wikidata.org/prop/direct/P1299'), ('label', 'depicted by')]
[('property', 'http://www.wikidata.org/prop/direct/P1343'), ('label', 'described by source')]
[('property', 'http://www.wikidata.org/prop/direct/P135'), ('label', 'movement')]
[('property', 'http://www.wikidata.org/prop/direct/P140'), ('label', 'religion')]
[('property', 'http://www.wikidata.org/prop/direct/P1412'), ('label', 'languages spoken, written or signed')]
[('property', 

### Get all the classes connected to "Dante Alighieri" (wd:Q1067)

In [29]:
queryString = """
SELECT DISTINCT ?objectClass ?label
WHERE {   
   wd:Q1067 ?property ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q16695773'), ('label', 'WikiProject')]
[('objectClass', 'http://www.wikidata.org/entity/Q186165'), ('label', 'web portal')]
[('objectClass', 'http://www.wikidata.org/entity/Q28653'), ('label', 'mendicant order')]
[('objectClass', 'http://www.wikidata.org/entity/Q15893266'), ('label', 'former entity')]
[('objectClass', 'http://www.wikidata.org/entity/Q170161'), ('label', 'guild')]
[('objectClass', 'http://www.wikidata.org/entity/Q879146'), ('label', 'Christian denomination')]
[('objectClass', 'http://www.wikidata.org/entity/Q48264'), ('label', 'gender identity')]
[('objectClass', 'http://www.wikidata.org/entity/Q56165908'), ('label', 'Jewish encyclopedia')]
[('objectClass', 'http://www.wikidata.org/entity/Q12737077'), ('label', 'occupation')]
[('objectClass', 'http://www.wikidata.org/entity/Q1288568'), ('label', 'modern language')]
[('objectClass', 'http://www.wikidata.org/entity/Q13406463'), ('label', 'Wikimedia list article')]
[

### Get all the couples (object property - list of the classes of the objects that are connected to "Dante Alighieri" (wd:Q1067) through that object property)

In [5]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?objectClass ; separator = ",")  AS ?objectsClasses) (GROUP_CONCAT(?labelObjectClass ; separator = ",")  AS ?labelsObjectsClasses)
WHERE 
{   
   wd:Q1067 ?property ?object .  
   ?object wdt:P31 ?objectClass .
   
   # Labels
   ?property sc:name ?labelProperty .      
   ?objectClass sc:name ?labelObjectClass .
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P735'), ('labelProperty', 'given name'), ('objectsClasses', 'http://www.wikidata.org/entity/Q12308941'), ('labelsObjectsClasses', 'male given name')]
[('property', 'http://www.wikidata.org/prop/direct/P135'), ('labelProperty', 'movement'), ('objectsClasses', 'http://www.wikidata.org/entity/Q3326717'), ('labelsObjectsClasses', 'literary movement')]
[('property', 'http://www.wikidata.org/prop/direct/P106'), ('labelProperty', 'occupation'), ('objectsClasses', 'http://www.wikidata.org/entity/Q12737077,http://www.wikidata.org/entity/Q12737077,http://www.wikidata.org/entity/Q12737077,http://www.wikidata.org/entity/Q28640,http://www.wikidata.org/entity/Q28640,http://www.wikidata.org/entity/Q28640,http://www.wikidata.org/entity/Q28640,http://www.wikidata.org/entity/Q28640,http://www.wikidata.org/entity/Q28640,http://www.wikidata.org/entity/Q28640,http://www.wikidata.org/entity/Q4164871,http://www.wikidata.org/entity/Q8434,http://www.wi

### Get all the couples (object property - list of the  objects that are connected to "Dante Alighieri"  (wd:Q1067) through that object property)

In [56]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?object ; separator = ",")  AS ?objects) (GROUP_CONCAT(?labelObject; separator = ",")  AS ?labelsObjects)
WHERE 
{   
   wd:Q1067 ?property ?object .  
 
   # Labels
   ?property sc:name ?labelProperty .      
   ?object sc:name ?labelObject
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P735'), ('labelProperty', 'given name'), ('objects', 'http://www.wikidata.org/entity/Q18182706'), ('labelsObjects', 'Dante')]
[('property', 'http://www.wikidata.org/prop/direct/P135'), ('labelProperty', 'movement'), ('objects', 'http://www.wikidata.org/entity/Q542036'), ('labelsObjects', 'Dolce Stil Novo')]
[('property', 'http://www.wikidata.org/prop/direct/P106'), ('labelProperty', 'occupation'), ('objects', 'http://www.wikidata.org/entity/Q12144794,http://www.wikidata.org/entity/Q15994177,http://www.wikidata.org/entity/Q36180,http://www.wikidata.org/entity/Q482980,http://www.wikidata.org/entity/Q4964182,http://www.wikidata.org/entity/Q49757,http://www.wikidata.org/entity/Q822146,http://www.wikidata.org/entity/Q82955,http://www.wikidata.org/entity/Q58968'), ('labelsObjects', 'prosaist,political theorist,writer,author,philosopher,poet,lyricist,politician,intellectual')]
[('property', 'http://www.wikidata.org/prop/direct/P1196')

### Final Query

Final query for this task

In [31]:
queryString = """
SELECT DISTINCT ?occupation ?label
WHERE {   
   wd:Q1067 wdt:P106 ?occupation .
   ?occupation sc:name ?label .
}
"""
print("Results")
x=run_query(queryString)

Results
[('occupation', 'http://www.wikidata.org/entity/Q12144794'), ('label', 'prosaist')]
[('occupation', 'http://www.wikidata.org/entity/Q15994177'), ('label', 'political theorist')]
[('occupation', 'http://www.wikidata.org/entity/Q36180'), ('label', 'writer')]
[('occupation', 'http://www.wikidata.org/entity/Q482980'), ('label', 'author')]
[('occupation', 'http://www.wikidata.org/entity/Q4964182'), ('label', 'philosopher')]
[('occupation', 'http://www.wikidata.org/entity/Q49757'), ('label', 'poet')]
[('occupation', 'http://www.wikidata.org/entity/Q822146'), ('label', 'lyricist')]
[('occupation', 'http://www.wikidata.org/entity/Q82955'), ('label', 'politician')]
[('occupation', 'http://www.wikidata.org/entity/Q58968'), ('label', 'intellectual')]
9


## Task 3
Identify the BGP for literary movement

### Get all data property of "poet" (wd:Q49757)

In [32]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q49757 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1014'), ('label', 'Art & Architecture Thesaurus ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1296'), ('label', 'Gran Enciclopèdia Catalana ID')]
[('property', 'http://www.wikidata.org/prop/direct/P227'), ('label', 'GND ID')]
[('property', 'http://www.wikidata.org/prop/direct/P244'), ('label', 'Library of Congress authority ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2581'), ('label', 'BabelNet ID')]
[('property', 'http://www.wikidata.org/prop/direct/P268'), ('label', 'Bibliothèque nationale de France ID')]
[('property', 'http://www.wikidata.org/prop/direct/P3417'), ('label', 'Quora topic ID')]
[('property', 'http://www.wikidata.org/prop/direct/P349'), ('label', 'National Diet Library ID')]
[('property', 'http://www.wikidata.org/prop/direct/P3553'), ('label', 'Zhihu topic ID')]
[('property', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('property', 'http://www.wikidata.

### Get all data property of "poet" (wd:Q49757) that do not contain ID

In [33]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q49757 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .
   FILTER REGEX(?label, "^((?!ID).)*$") . 
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('property', 'http://www.wikidata.org/prop/direct/P867'), ('label', 'ROME Occupation Code (v3)')]
2


### Get the class of "poet" (wd:Q49757)

In [17]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q49757 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q28640'), ('label', 'profession')]
1


### Get all object properties of "poet" (wd:Q49757)

In [34]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q49757 ?property ?object .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1343'), ('label', 'described by source')]
[('property', 'http://www.wikidata.org/prop/direct/P1709'), ('label', 'equivalent class')]
[('property', 'http://www.wikidata.org/prop/direct/P1889'), ('label', 'different from')]
[('property', 'http://www.wikidata.org/prop/direct/P1963'), ('label', 'properties for this type')]
[('property', 'http://www.wikidata.org/prop/direct/P279'), ('label', 'subclass of')]
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('label', 'instance of')]
[('property', 'http://www.wikidata.org/prop/direct/P425'), ('label', 'field of this occupation')]
[('property', 'http://www.wikidata.org/prop/direct/P460'), ('label', 'said to be the same as')]
[('property', 'http://www.wikidata.org/prop/direct/P6186'), ('label', 'category for eponymous categories')]
[('property', 'http://www.wikidata.org/prop/direct/P910'), ('label', "topic's main category")]
10


### Get all the classes connected to "poet" (wd:Q49757)

In [4]:
queryString = """
SELECT DISTINCT ?objectClass ?label
WHERE {   
   wd:Q49757 ?property ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q12737077'), ('label', 'occupation')]
[('objectClass', 'http://www.wikidata.org/entity/Q28640'), ('label', 'profession')]
[('objectClass', 'http://www.wikidata.org/entity/Q30432511'), ('label', 'metacategory in Wikimedia projects')]
[('objectClass', 'http://www.wikidata.org/entity/Q4164871'), ('label', 'position')]
[('objectClass', 'http://www.wikidata.org/entity/Q4167836'), ('label', 'Wikimedia category')]
[('objectClass', 'http://www.wikidata.org/entity/Q4263830'), ('label', 'literary form')]
[('objectClass', 'http://www.wikidata.org/entity/Q7094076'), ('label', 'online database')]
[('objectClass', 'http://www.wikidata.org/entity/Q223393'), ('label', 'literary genre')]
[('objectClass', 'http://www.wikidata.org/entity/Q24017414'), ('label', 'second-order class')]
[('objectClass', 'http://www.wikidata.org/entity/Q1469824'), ('label', 'controlled vocabulary')]
[('objectClass', 'http://www.wikidata.org/entity/Q5185279'), ('label', 

### Get all the couples (object property - list of the classes of the objects that are connected to "poet" (wd:Q49757) through that object property)

In [6]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?objectClass ; separator = ",")  AS ?objectsClasses) (GROUP_CONCAT(?labelObjectClass ; separator = ",")  AS ?labelsObjectsClasses)
WHERE 
{   
   wd:Q49757 ?property ?object .  
   ?object wdt:P31 ?objectClass .
   
   # Labels
   ?property sc:name ?labelProperty .      
   ?objectClass sc:name ?labelObjectClass .
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P6186'), ('labelProperty', 'category for eponymous categories'), ('objectsClasses', 'http://www.wikidata.org/entity/Q30432511'), ('labelsObjectsClasses', 'metacategory in Wikimedia projects')]
[('property', 'http://www.wikidata.org/prop/direct/P425'), ('labelProperty', 'field of this occupation'), ('objectsClasses', 'http://www.wikidata.org/entity/Q4263830,http://www.wikidata.org/entity/Q223393'), ('labelsObjectsClasses', 'literary form,literary genre')]
[('property', 'http://www.wikidata.org/prop/direct/P460'), ('labelProperty', 'said to be the same as'), ('objectsClasses', 'http://www.wikidata.org/entity/Q28640'), ('labelsObjectsClasses', 'profession')]
[('property', 'http://www.wikidata.org/prop/direct/P1889'), ('labelProperty', 'different from'), ('objectsClasses', 'http://www.wikidata.org/entity/Q5185279'), ('labelsObjectsClasses', 'poem')]
[('property', 'http://www.wikidata.org/prop/direct/P1963'), ('labelProperty', 'prop

### Get all the couples (object property - list of the  objects that are connected to "poet" (wd:Q49757) through that object property)

In [57]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?object ; separator = ",")  AS ?objects) (GROUP_CONCAT(?labelObject; separator = ",")  AS ?labelsObjects)
WHERE 
{   
   wd:Q49757 ?property ?object .  
 
   # Labels
   ?property sc:name ?labelProperty .      
   ?object sc:name ?labelObject
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P6186'), ('labelProperty', 'category for eponymous categories'), ('objects', 'http://www.wikidata.org/entity/Q9579555'), ('labelsObjects', 'Category:Wikipedia categories named after poets')]
[('property', 'http://www.wikidata.org/prop/direct/P425'), ('labelProperty', 'field of this occupation'), ('objects', 'http://www.wikidata.org/entity/Q482'), ('labelsObjects', 'poetry')]
[('property', 'http://www.wikidata.org/prop/direct/P460'), ('labelProperty', 'said to be the same as'), ('objects', 'http://www.wikidata.org/entity/Q1293108'), ('labelsObjects', 'Dichter')]
[('property', 'http://www.wikidata.org/prop/direct/P1963'), ('labelProperty', 'properties for this type'), ('objects', 'http://www.wikidata.org/entity/P5509,http://www.wikidata.org/entity/P5477,http://www.wikidata.org/entity/P5506,http://www.wikidata.org/entity/P5710,http://www.wikidata.org/entity/P5430,http://www.wikidata.org/entity/P5343,http://www.wikidata.org/entity/

### Final Query

Final query for this task

In [58]:
queryString = """
SELECT DISTINCT ?objectClass ?labelobjectClass
WHERE {   
   wd:Q1067 wdt:P135 ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?labelobjectClass .
}LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q3326717'), ('labelobjectClass', 'literary movement')]
1


## Task 4
Return the poets born in the same place of the author of The Divine Comedy who are born between 1300 and 1500 (1300 and 1500 included) (the result set must be a list of couples poets IRI and label).

### Get the first 25 "date of birth" (wdt:P569) of all persons whose occupation (wdt:P106) is poet (wd:Q49757)

In [69]:
queryString = """
SELECT DISTINCT ?poet ?labelPoet ?dateOfBirth
WHERE {   
    ?poet wdt:P106 wd:Q49757 ;
          wdt:P569 ?dateOfBirth ;
          sc:name ?labelPoet .
}LIMIT 25
"""
print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q1373516'), ('labelPoet', 'Eugénio de Castro'), ('dateOfBirth', '1869-03-04T00:00:00Z')]
[('poet', 'http://www.wikidata.org/entity/Q589406'), ('labelPoet', 'Empress Xiaoquancheng'), ('dateOfBirth', '1808-03-24T00:00:00Z')]
[('poet', 'http://www.wikidata.org/entity/Q610608'), ('labelPoet', 'António Aleixo'), ('dateOfBirth', '1899-02-18T00:00:00Z')]
[('poet', 'http://www.wikidata.org/entity/Q710504'), ('labelPoet', 'Bohuslav Balbín'), ('dateOfBirth', '1621-12-03T00:00:00Z')]
[('poet', 'http://www.wikidata.org/entity/Q8188119'), ('labelPoet', 'Adelina Lopes Vieira'), ('dateOfBirth', '1850-09-20T00:00:00Z')]
[('poet', 'http://www.wikidata.org/entity/Q8201526'), ('labelPoet', 'Antonio Márquez Salas'), ('dateOfBirth', '1919-07-06T00:00:00Z')]
[('poet', 'http://www.wikidata.org/entity/Q8201629'), ('labelPoet', 'Antonio Rendic'), ('dateOfBirth', '1896-12-20T00:00:00Z')]
[('poet', 'http://www.wikidata.org/entity/Q8202094'), ('labelPoet', 'Antôni

### Get the first 25 persons whose occupation (wdt:P106) is poet (wd:Q49757) and has "date of birth" (wdt:P569) between 1300 and 1500 

In [78]:
queryString = """
SELECT DISTINCT ?poet ?labelPoet ?yearOfBirth
WHERE {   
    ?poet wdt:P106 wd:Q49757 ;
          wdt:P569 ?dateOfBirth ;
          sc:name ?labelPoet .
    BIND(SUBSTR(STR(?dateOfBirth),0,4) AS ?yearOfBirth)
    FILTER (?yearOfBirth >= "1300" && ?yearOfBirth <= "1500")
}LIMIT 25
"""
print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q16075667'), ('labelPoet', 'Du Mu'), ('yearOfBirth', '1458')]
[('poet', 'http://www.wikidata.org/entity/Q16039483'), ('labelPoet', 'Arias Barbosa'), ('yearOfBirth', '1450')]
[('poet', 'http://www.wikidata.org/entity/Q16039483'), ('labelPoet', 'Arias Barbosa'), ('yearOfBirth', '1470')]
[('poet', 'http://www.wikidata.org/entity/Q562300'), ('labelPoet', 'Gilles Joye'), ('yearOfBirth', '1424')]
[('poet', 'http://www.wikidata.org/entity/Q655618'), ('labelPoet', 'Francesco Berni'), ('yearOfBirth', '1497')]
[('poet', 'http://www.wikidata.org/entity/Q296272'), ('labelPoet', 'Pietro Aretino'), ('yearOfBirth', '1492')]
[('poet', 'http://www.wikidata.org/entity/Q5880389'), ('labelPoet', 'Girolamo Casio'), ('yearOfBirth', '1464')]
[('poet', 'http://www.wikidata.org/entity/Q21003797'), ('labelPoet', 'Michel Verin'), ('yearOfBirth', '1468')]
[('poet', 'http://www.wikidata.org/entity/Q22586308'), ('labelPoet', 'Domenico da Prato'), ('yearOfBirth', '13

### Final Query

Final query for this task

In [81]:
queryString = """
SELECT DISTINCT ?poet ?labelPoet
WHERE {   
    #Dante place of birth
    wd:Q1067 wdt:P19 ?placeOfBirthDante .
    
    #All poets born between 1300 and 1500
    ?poet wdt:P106 wd:Q49757 ;
          wdt:P569 ?dateOfBirth ;
          wdt:P19 ?placeOfBirth ;
          sc:name ?labelPoet .
    BIND(SUBSTR(STR(?dateOfBirth),0,4) AS ?yearOfBirth) .
    FILTER (?yearOfBirth >= "1300" && ?yearOfBirth <= "1500" && ?placeOfBirth = ?placeOfBirthDante)
}LIMIT 25
"""
print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q21003797'), ('labelPoet', 'Michel Verin')]
[('poet', 'http://www.wikidata.org/entity/Q1248954'), ('labelPoet', 'Giovanni di Bernardo Rucellai')]
[('poet', 'http://www.wikidata.org/entity/Q177854'), ('labelPoet', "Lorenzo de' Medici")]
[('poet', 'http://www.wikidata.org/entity/Q328117'), ('labelPoet', 'Girolamo Benivieni')]
[('poet', 'http://www.wikidata.org/entity/Q721846'), ('labelPoet', 'Luigi Alamanni')]
[('poet', 'http://www.wikidata.org/entity/Q1399'), ('labelPoet', 'Niccolò Machiavelli')]
[('poet', 'http://www.wikidata.org/entity/Q366499'), ('labelPoet', 'Orcagna')]
[('poet', 'http://www.wikidata.org/entity/Q1402'), ('labelPoet', 'Giovanni Boccaccio')]
[('poet', 'http://www.wikidata.org/entity/Q631635'), ('labelPoet', 'Luigi Pulci')]
[('poet', 'http://www.wikidata.org/entity/Q2857170'), ('labelPoet', 'Antonio Alamanni')]
[('poet', 'http://www.wikidata.org/entity/Q1527917'), ('labelPoet', 'Giuliano Dati')]
[('poet', 'http://www.wi

## Task 5
Identify the BGP for literary work

### Get the objects connected to "Dante Alighieri" (wd:Q1067) with "described by source" (wdt:P1343) property

In [18]:
queryString = """
SELECT DISTINCT ?object ?label
WHERE {   
   wd:Q1067 wdt:P1343 ?object .
   ?object sc:name ?label .
}LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q19180675'), ('label', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('object', 'http://www.wikidata.org/entity/Q20078554'), ('label', 'Great Soviet Encyclopedia (1926–1947)')]
[('object', 'http://www.wikidata.org/entity/Q2041543'), ('label', 'Ottův slovník naučný')]
[('object', 'http://www.wikidata.org/entity/Q20743760'), ('label', 'Encyclopedia of Political Theory (2010 ed.)')]
[('object', 'http://www.wikidata.org/entity/Q302556'), ('label', 'Catholic Encyclopedia')]
[('object', 'http://www.wikidata.org/entity/Q3181656'), ('label', 'The Nuttall Encyclopædia')]
[('object', 'http://www.wikidata.org/entity/Q4173137'), ('label', 'Jewish Encyclopedia of Brockhaus and Efron')]
[('object', 'http://www.wikidata.org/entity/Q51955019'), ('label', 'BEIC Digital Library')]
[('object', 'http://www.wikidata.org/entity/Q602358'), ('label', 'Brockhaus and Efron Encyclopedic Dictionary')]
[('object', 'http://www.wikidata.org/entity/Q6731152

### Get the objects connected to "Dante Alighieri" (wd:Q1067) with "notable work" (wdt:P800) property

In [19]:
queryString = """
SELECT DISTINCT ?object ?label
WHERE {   
   wd:Q1067 wdt:P800 ?object .
   ?object sc:name ?label .
}LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q18084'), ('label', 'Vita Nuova')]
[('object', 'http://www.wikidata.org/entity/Q40185'), ('label', 'The Divine Comedy')]
[('object', 'http://www.wikidata.org/entity/Q134221'), ('label', 'De Monarchia')]
[('object', 'http://www.wikidata.org/entity/Q18081'), ('label', 'De vulgari eloquentia')]
[('object', 'http://www.wikidata.org/entity/Q666014'), ('label', 'Convivio')]
5


### Final Query

Final query for this task

In [20]:
queryString = """
SELECT DISTINCT ?objectClass ?labelObjectClass
WHERE {   
   wd:Q1067 wdt:P800 ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?labelObjectClass .
}LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q47461344'), ('labelObjectClass', 'written work')]
[('objectClass', 'http://www.wikidata.org/entity/Q7725634'), ('labelObjectClass', 'literary work')]
2


## Task 6
Compare the author of The Divine Comedy with Giovanni Boccaccio. Return for these poets the number of literary work they wrote (the result set must be two triples -one for each author- author IRI, label and #literary work).

### Get "given name" (wdt:P735) and "family name" (wdt:P734) of "Dante Alighieri" (wd:Q1067)

In [101]:
queryString = """
SELECT DISTINCT ?labelDante ?givenNameDante ?labelGivenNameDante ?familyNameDante ?labelFamilyNameDante
WHERE {   
    wd:Q1067 wdt:P735 ?givenNameDante ;
            wdt:P734 ?familyNameDante ;
            sc:name ?labelDante .
    ?givenNameDante sc:name ?labelGivenNameDante .
    ?familyNameDante sc:name ?labelFamilyNameDante .    
}LIMIT 25
"""
print("Results")
x=run_query(queryString)

Results
[('labelDante', 'Dante Alighieri'), ('givenNameDante', 'http://www.wikidata.org/entity/Q18182706'), ('labelGivenNameDante', 'Dante'), ('familyNameDante', 'http://www.wikidata.org/entity/Q15782426'), ('labelFamilyNameDante', 'Alighieri')]
1


### Get the URI of "Giovanni Boccaccio"

In [102]:
queryString = """
SELECT DISTINCT ?poet ?labelHuman 
WHERE {   
    ?poet  wdt:P106 wd:Q49757 ;
           sc:name ?labelHuman .   
    FILTER (STR(?labelHuman) = "Giovanni Boccaccio") .
}LIMIT 25
"""
print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q1402'), ('labelHuman', 'Giovanni Boccaccio')]
1


### Get all "literary work" (wd:Q7725634) of "Dante Alighieri" (wd:Q1067)

In [107]:
queryString = """
SELECT DISTINCT ?literaryWork ?labelLiteraryWork
WHERE {       
    wd:Q1067 wdt:P800 ?literaryWork .
    ?literaryWork wdt:P31 wd:Q7725634 ;
                  sc:name ?labelLiteraryWork .        
}LIMIT 25
"""
print("Results")
x=run_query(queryString)

Results
[('literaryWork', 'http://www.wikidata.org/entity/Q18084'), ('labelLiteraryWork', 'Vita Nuova')]
[('literaryWork', 'http://www.wikidata.org/entity/Q134221'), ('labelLiteraryWork', 'De Monarchia')]
[('literaryWork', 'http://www.wikidata.org/entity/Q18081'), ('labelLiteraryWork', 'De vulgari eloquentia')]
[('literaryWork', 'http://www.wikidata.org/entity/Q666014'), ('labelLiteraryWork', 'Convivio')]
4


### Final Query

Final query for this task

In [110]:
queryString = """
SELECT ?author ?labelAuthor (COUNT (DISTINCT ?literaryWork) AS ?numberOfLiteraryWorks)
WHERE { 
    ?author wdt:P800 ?literaryWork ;
            sc:name ?labelAuthor .
    ?literaryWork wdt:P31 wd:Q7725634  . 
    FILTER (?author = wd:Q1067 || ?author = wd:Q1402)            
}GROUP BY ?author ?labelAuthor
"""
print("Results")
x=run_query(queryString)

Results
[('author', 'http://www.wikidata.org/entity/Q1402'), ('labelAuthor', 'Giovanni Boccaccio'), ('numberOfLiteraryWorks', '2')]
[('author', 'http://www.wikidata.org/entity/Q1067'), ('labelAuthor', 'Dante Alighieri'), ('numberOfLiteraryWorks', '4')]
2


## Task 7
Consider italian poets, their place of birth (the city) and their university education. Return for each city the number of poets who born there and who was educated there (the result set must be a list of triples city IRI, label and #poets).

### Get all poet (wd:Q49757) with year of "date of birth" (wdt:P569) grater than 1850

In [114]:
queryString = """
SELECT ?poet ?labelPoet ?yearOfBirth
WHERE { 
    ?poet wdt:P106 wd:Q49757 ;
          wdt:P569 ?dateOfBirth ;
          sc:name ?labelPoet .
    BIND(SUBSTR(STR(?dateOfBirth),0,4) AS ?yearOfBirth) .
    FILTER (?yearOfBirth >= "1850")
}LIMIT 25
"""
print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q1373516'), ('labelPoet', 'Eugénio de Castro'), ('yearOfBirth', '1869')]
[('poet', 'http://www.wikidata.org/entity/Q610608'), ('labelPoet', 'António Aleixo'), ('yearOfBirth', '1899')]
[('poet', 'http://www.wikidata.org/entity/Q8188119'), ('labelPoet', 'Adelina Lopes Vieira'), ('yearOfBirth', '1850')]
[('poet', 'http://www.wikidata.org/entity/Q8201526'), ('labelPoet', 'Antonio Márquez Salas'), ('yearOfBirth', '1919')]
[('poet', 'http://www.wikidata.org/entity/Q8201629'), ('labelPoet', 'Antonio Rendic'), ('yearOfBirth', '1896')]
[('poet', 'http://www.wikidata.org/entity/Q8202094'), ('labelPoet', 'Antônio de Castro Assumpção'), ('yearOfBirth', '1922')]
[('poet', 'http://www.wikidata.org/entity/Q1191225'), ('labelPoet', 'Tan Sitong'), ('yearOfBirth', '1865')]
[('poet', 'http://www.wikidata.org/entity/Q4790571'), ('labelPoet', 'Arifin C. Noer'), ('yearOfBirth', '1941')]
[('poet', 'http://www.wikidata.org/entity/Q6143494'), ('labelPoet', 'Lin

### Get all data property of "Eugénio de Castro" (wd:Q1373516)

In [116]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q1373516 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P9053'), ('label', 'Projecto Vercial author ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1005'), ('label', 'Portuguese National Library ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1006'), ('label', 'Nationale Thesaurus voor Auteurs ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1273'), ('label', 'CANTIC ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1296'), ('label', 'Gran Enciclopèdia Catalana ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1417'), ('label', 'Encyclopædia Britannica Online ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1580'), ('label', 'University of Barcelona authority ID')]
[('property', 'http://www.wikidata.org/prop/direct/P213'), ('label', 'ISNI')]
[('property', 'http://www.wikidata.org/prop/direct/P214'), ('label', 'VIAF ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2163'), ('label', 'FAST ID')]
[('property', 'http://www.wi

### Get all data property of "Eugénio de Castro"  (wd:Q1373516) that do not contain ID

In [117]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q1373516 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .
   FILTER REGEX(?label, "^((?!ID).)*$") . 
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P213'), ('label', 'ISNI')]
[('property', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('property', 'http://www.wikidata.org/prop/direct/P569'), ('label', 'date of birth')]
[('property', 'http://www.wikidata.org/prop/direct/P570'), ('label', 'date of death')]
4


### Get the class of "Eugénio de Castro" (wd:Q1373516)

In [119]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q1373516 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q5'), ('label', 'human')]
1


### Get all object properties of "Eugénio de Castro"  (wd:Q1373516)

In [120]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q1373516 ?property ?object .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P106'), ('label', 'occupation')]
[('property', 'http://www.wikidata.org/prop/direct/P1343'), ('label', 'described by source')]
[('property', 'http://www.wikidata.org/prop/direct/P1412'), ('label', 'languages spoken, written or signed')]
[('property', 'http://www.wikidata.org/prop/direct/P166'), ('label', 'award received')]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P19'), ('label', 'place of birth')]
[('property', 'http://www.wikidata.org/prop/direct/P20'), ('label', 'place of death')]
[('property', 'http://www.wikidata.org/prop/direct/P21'), ('label', 'sex or gender')]
[('property', 'http://www.wikidata.org/prop/direct/P27'), ('label', 'country of citizenship')]
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('label', 'instance of')]
[('property', 'http://www.wikidata.org/prop/direct/P463'), ('label', 'member of')]
[('property', 'h

### Get all the classes connected to "Eugénio de Castro"  (wd:Q1373516)

In [121]:
queryString = """
SELECT DISTINCT ?objectClass ?label
WHERE {   
   wd:Q1373516 ?property ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q186165'), ('label', 'web portal')]
[('objectClass', 'http://www.wikidata.org/entity/Q1966910'), ('label', 'national academy')]
[('objectClass', 'http://www.wikidata.org/entity/Q48264'), ('label', 'gender identity')]
[('objectClass', 'http://www.wikidata.org/entity/Q12737077'), ('label', 'occupation')]
[('objectClass', 'http://www.wikidata.org/entity/Q1288568'), ('label', 'modern language')]
[('objectClass', 'http://www.wikidata.org/entity/Q13217644'), ('label', 'municipality of Portugal')]
[('objectClass', 'http://www.wikidata.org/entity/Q1549591'), ('label', 'big city')]
[('objectClass', 'http://www.wikidata.org/entity/Q15647906'), ('label', 'city of Portugal')]
[('objectClass', 'http://www.wikidata.org/entity/Q20181813'), ('label', 'colonial power')]
[('objectClass', 'http://www.wikidata.org/entity/Q28640'), ('label', 'profession')]
[('objectClass', 'http://www.wikidata.org/entity/Q34770'), ('label', 'language')]
[('objectClas

### Get all the couples (object property - list of the classes of the objects that are connected to "Eugénio de Castro"  (wd:Q1373516) through that object property)

In [122]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?objectClass ; separator = ",")  AS ?objectsClasses) (GROUP_CONCAT(?labelObjectClass ; separator = ",")  AS ?labelsObjectsClasses)
WHERE 
{   
   wd:Q1373516 ?property ?object .  
   ?object wdt:P31 ?objectClass .
   
   # Labels
   ?property sc:name ?labelProperty .      
   ?objectClass sc:name ?labelObjectClass .
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P735'), ('labelProperty', 'given name'), ('objectsClasses', 'http://www.wikidata.org/entity/Q12308941'), ('labelsObjectsClasses', 'male given name')]
[('property', 'http://www.wikidata.org/prop/direct/P106'), ('labelProperty', 'occupation'), ('objectsClasses', 'http://www.wikidata.org/entity/Q12737077,http://www.wikidata.org/entity/Q28640,http://www.wikidata.org/entity/Q28640,http://www.wikidata.org/entity/Q28640,http://www.wikidata.org/entity/Q4164871'), ('labelsObjectsClasses', 'occupation,profession,profession,profession,position')]
[('property', 'http://www.wikidata.org/prop/direct/P69'), ('labelProperty', 'educated at'), ('objectsClasses', 'http://www.wikidata.org/entity/Q45400320,http://www.wikidata.org/entity/Q875538'), ('labelsObjectsClasses', 'open-access publisher,public university')]
[('property', 'http://www.wikidata.org/prop/direct/P21'), ('labelProperty', 'sex or gender'), ('objectsClasses', 'http://www.wikidata.o

### Get all the couples (object property - list of the  objects that are connected to "Eugénio de Castro" (wd:Q1373516) through that object property)

In [115]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?object ; separator = ",")  AS ?objects) (GROUP_CONCAT(?labelObject; separator = ",")  AS ?labelsObjects)
WHERE 
{   
   wd:Q1373516 ?property ?object .  
 
   # Labels
   ?property sc:name ?labelProperty .      
   ?object sc:name ?labelObject
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P735'), ('labelProperty', 'given name'), ('objects', 'http://www.wikidata.org/entity/Q21286216'), ('labelsObjects', 'Eugénio')]
[('property', 'http://www.wikidata.org/prop/direct/P106'), ('labelProperty', 'occupation'), ('objects', 'http://www.wikidata.org/entity/Q1231865,http://www.wikidata.org/entity/Q36180,http://www.wikidata.org/entity/Q49757'), ('labelsObjects', 'pedagogue,writer,poet')]
[('property', 'http://www.wikidata.org/prop/direct/P69'), ('labelProperty', 'educated at'), ('objects', 'http://www.wikidata.org/entity/Q368643'), ('labelsObjects', 'University of Coimbra')]
[('property', 'http://www.wikidata.org/prop/direct/P21'), ('labelProperty', 'sex or gender'), ('objects', 'http://www.wikidata.org/entity/Q6581097'), ('labelsObjects', 'male')]
[('property', 'http://www.wikidata.org/prop/direct/P1412'), ('labelProperty', 'languages spoken, written or signed'), ('objects', 'http://www.wikidata.org/entity/Q5146'), ('labe

### Get the superclass of "public university" (wd:Q875538)

In [123]:
queryString = """
SELECT ?superClass ?labelSuperClass
WHERE 
{   
    wd:Q875538 wdt:P279 ?superClass .
    ?superClass sc:name ?labelSuperClass .      
}
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('superClass', 'http://www.wikidata.org/entity/Q23002037'), ('labelSuperClass', 'public educational institution')]
[('superClass', 'http://www.wikidata.org/entity/Q3918'), ('labelSuperClass', 'university')]
2


### Get all data property of "University of Coimbra" (wd:Q368643)

In [125]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q368643 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1417'), ('label', 'Encyclopædia Britannica Online ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1612'), ('label', 'Commons Institution page')]
[('property', 'http://www.wikidata.org/prop/direct/P2002'), ('label', 'Twitter username')]
[('property', 'http://www.wikidata.org/prop/direct/P2003'), ('label', 'Instagram username')]
[('property', 'http://www.wikidata.org/prop/direct/P2013'), ('label', 'Facebook ID')]
[('property', 'http://www.wikidata.org/prop/direct/P213'), ('label', 'ISNI')]
[('property', 'http://www.wikidata.org/prop/direct/P214'), ('label', 'VIAF ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2174'), ('label', 'Museum of Modern Art artist ID')]
[('property', 'http://www.wikidata.org/prop/direct/P227'), ('label', 'GND ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2427'), ('label', 'GRID ID')]
[('property', 'http://www.wikidata.org/prop/direct/P244'), ('label', 'Library of Congress

### Get all data property of "University of Coimbra"  (wd:Q368643) that do not contain ID

In [126]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q368643 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .
   FILTER REGEX(?label, "^((?!ID).)*$") . 
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1612'), ('label', 'Commons Institution page')]
[('property', 'http://www.wikidata.org/prop/direct/P2002'), ('label', 'Twitter username')]
[('property', 'http://www.wikidata.org/prop/direct/P2003'), ('label', 'Instagram username')]
[('property', 'http://www.wikidata.org/prop/direct/P213'), ('label', 'ISNI')]
[('property', 'http://www.wikidata.org/prop/direct/P3608'), ('label', 'EU VAT number')]
[('property', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('property', 'http://www.wikidata.org/prop/direct/P3793'), ('label', 'IPv6 routing prefix')]
[('property', 'http://www.wikidata.org/prop/direct/P571'), ('label', 'inception')]
[('property', 'http://www.wikidata.org/prop/direct/P8687'), ('label', 'social media followers')]
9


### Get the class of "University of Coimbra" (wd:Q368643)

In [127]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q368643 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q45400320'), ('label', 'open-access publisher')]
[('class', 'http://www.wikidata.org/entity/Q875538'), ('label', 'public university')]
2


### Get all object properties of "University of Coimbra"  (wd:Q368643)

In [128]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q368643 ?property ?object .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P112'), ('label', 'founded by')]
[('property', 'http://www.wikidata.org/prop/direct/P131'), ('label', 'located in the administrative territorial entity')]
[('property', 'http://www.wikidata.org/prop/direct/P1416'), ('label', 'affiliation')]
[('property', 'http://www.wikidata.org/prop/direct/P154'), ('label', 'logo image')]
[('property', 'http://www.wikidata.org/prop/direct/P159'), ('label', 'headquarters location')]
[('property', 'http://www.wikidata.org/prop/direct/P166'), ('label', 'award received')]
[('property', 'http://www.wikidata.org/prop/direct/P17'), ('label', 'country')]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P27'), ('label', 'country of citizenship')]
[('property', 'http://www.wikidata.org/prop/direct/P2936'), ('label', 'language used')]
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('label', 'instance of')]
[('prope

### Get all the classes connected to "University of Coimbra"  (wd:Q368643)

In [129]:
queryString = """
SELECT DISTINCT ?objectClass ?label
WHERE {   
   wd:Q368643 ?property ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q3457065'), ('label', 'university network')]
[('objectClass', 'http://www.wikidata.org/entity/Q19844914'), ('label', 'university building')]
[('objectClass', 'http://www.wikidata.org/entity/Q4671277'), ('label', 'academic institution')]
[('objectClass', 'http://www.wikidata.org/entity/Q618779'), ('label', 'award')]
[('objectClass', 'http://www.wikidata.org/entity/Q13226383'), ('label', 'facility')]
[('objectClass', 'http://www.wikidata.org/entity/Q1288568'), ('label', 'modern language')]
[('objectClass', 'http://www.wikidata.org/entity/Q13217644'), ('label', 'municipality of Portugal')]
[('objectClass', 'http://www.wikidata.org/entity/Q1549591'), ('label', 'big city')]
[('objectClass', 'http://www.wikidata.org/entity/Q15647906'), ('label', 'city of Portugal')]
[('objectClass', 'http://www.wikidata.org/entity/Q20181813'), ('label', 'colonial power')]
[('objectClass', 'http://www.wikidata.org/entity/Q34770'), ('label', 'language')]

### Get all the couples (object property - list of the classes of the objects that are connected to "University of Coimbra"  (wd:Q368643) through that object property)

In [130]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?objectClass ; separator = ",")  AS ?objectsClasses) (GROUP_CONCAT(?labelObjectClass ; separator = ",")  AS ?labelsObjectsClasses)
WHERE 
{   
   wd:Q368643 ?property ?object .  
   ?object wdt:P31 ?objectClass .
   
   # Labels
   ?property sc:name ?labelProperty .      
   ?objectClass sc:name ?labelObjectClass .
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P17'), ('labelProperty', 'country'), ('objectsClasses', 'http://www.wikidata.org/entity/Q20181813,http://www.wikidata.org/entity/Q3624078,http://www.wikidata.org/entity/Q6256'), ('labelsObjectsClasses', 'colonial power,sovereign state,country')]
[('property', 'http://www.wikidata.org/prop/direct/P166'), ('labelProperty', 'award received'), ('objectsClasses', 'http://www.wikidata.org/entity/Q618779,http://www.wikidata.org/entity/Q60754876'), ('labelsObjectsClasses', 'award,grade of an order')]
[('property', 'http://www.wikidata.org/prop/direct/P8989'), ('labelProperty', 'category for the view of the item'), ('objectsClasses', 'http://www.wikidata.org/entity/Q4167836'), ('labelsObjectsClasses', 'Wikimedia category')]
[('property', 'http://www.wikidata.org/prop/direct/P8933'), ('labelProperty', 'category for the view from the item'), ('objectsClasses', 'http://www.wikidata.org/entity/Q4167836'), ('labelsObjectsClasses', 'Wikimedia

### Get all the couples (object property - list of the objects that are connected to "University of Coimbra" (wd:Q368643) through that object property)

In [124]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?object ; separator = ",")  AS ?objects) (GROUP_CONCAT(?labelObject; separator = ",")  AS ?labelsObjects)
WHERE 
{   
   wd:Q368643 ?property ?object .  
 
   # Labels
   ?property sc:name ?labelProperty .      
   ?object sc:name ?labelObject
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P17'), ('labelProperty', 'country'), ('objects', 'http://www.wikidata.org/entity/Q45'), ('labelsObjects', 'Portugal')]
[('property', 'http://www.wikidata.org/prop/direct/P166'), ('labelProperty', 'award received'), ('objects', 'http://www.wikidata.org/entity/Q20886648,http://www.wikidata.org/entity/Q3321940'), ('labelsObjects', 'Grand Cross of the Military Order of Saint James of the Sword,Princess of Asturias Award for International Cooperation')]
[('property', 'http://www.wikidata.org/prop/direct/P8989'), ('labelProperty', 'category for the view of the item'), ('objects', 'http://www.wikidata.org/entity/Q104600561'), ('labelsObjects', 'Category:Views of Universidade de Coimbra')]
[('property', 'http://www.wikidata.org/prop/direct/P8933'), ('labelProperty', 'category for the view from the item'), ('objects', 'http://www.wikidata.org/entity/Q104235388'), ('labelsObjects', 'Category:Views from Universidade de Coimbra')]
[('prope

### Get URI of "Italy"

In [131]:
queryString = """
SELECT DISTINCT ?nation ?labelNation
WHERE 
{   
    ?sbj wdt:P27 ?nation ;
         wdt:P31 wd:Q5 .
    ?nation sc:name ?labelNation .
    FILTER CONTAINS(?labelNation, "Italy") .
}
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('nation', 'http://www.wikidata.org/entity/Q172579'), ('labelNation', 'Kingdom of Italy')]
[('nation', 'http://www.wikidata.org/entity/Q38'), ('labelNation', 'Italy')]
[('nation', 'http://www.wikidata.org/entity/Q223936'), ('labelNation', 'Kingdom of Italy')]
[('nation', 'http://www.wikidata.org/entity/Q2268597'), ('labelNation', 'United Provinces of Central Italy')]
[('nation', 'http://www.wikidata.org/entity/Q838931'), ('labelNation', 'Kingdom of Italy')]
[('nation', 'http://www.wikidata.org/entity/Q1098499'), ('labelNation', 'Kingdom of Italy')]
[('nation', 'http://www.wikidata.org/entity/Q207989'), ('labelNation', 'Southern Italy')]
[('nation', 'http://www.wikidata.org/entity/Q913582'), ('labelNation', 'Roman Italy')]
[('nation', 'http://www.wikidata.org/entity/Q48742118'), ('labelNation', 'Kingdom of Italy under Fascism')]
[('nation', 'http://www.wikidata.org/entity/Q66477411'), ('labelNation', 'Italy')]
[('nation', 'http://www.wikidata.org/entity/Q2643575'), ('labelNatio

### Get all data property of "Florence" (wd:Q2044)

In [5]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q2044 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1036'), ('label', 'Dewey Decimal Classification')]
[('property', 'http://www.wikidata.org/prop/direct/P1082'), ('label', 'population')]
[('property', 'http://www.wikidata.org/prop/direct/P1225'), ('label', 'U.S. National Archives Identifier')]
[('property', 'http://www.wikidata.org/prop/direct/P1245'), ('label', 'OmegaWiki Defined Meaning')]
[('property', 'http://www.wikidata.org/prop/direct/P1296'), ('label', 'Gran Enciclopèdia Catalana ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1417'), ('label', 'Encyclopædia Britannica Online ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1438'), ('label', 'Jewish Encyclopedia ID (Russian)')]
[('property', 'http://www.wikidata.org/prop/direct/P1449'), ('label', 'nickname')]
[('property', 'http://www.wikidata.org/prop/direct/P1566'), ('label', 'GeoNames ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1667'), ('label', 'Getty Thesaurus of Geographic Names I

### Get all data property of "Florence"  (wd:Q2044) that do not contain ID

In [6]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q2044 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .
   FILTER REGEX(?label, "^((?!ID).)*$") . 
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1036'), ('label', 'Dewey Decimal Classification')]
[('property', 'http://www.wikidata.org/prop/direct/P1082'), ('label', 'population')]
[('property', 'http://www.wikidata.org/prop/direct/P1225'), ('label', 'U.S. National Archives Identifier')]
[('property', 'http://www.wikidata.org/prop/direct/P1245'), ('label', 'OmegaWiki Defined Meaning')]
[('property', 'http://www.wikidata.org/prop/direct/P1449'), ('label', 'nickname')]
[('property', 'http://www.wikidata.org/prop/direct/P1937'), ('label', 'UN/LOCODE')]
[('property', 'http://www.wikidata.org/prop/direct/P2044'), ('label', 'elevation above sea level')]
[('property', 'http://www.wikidata.org/prop/direct/P2046'), ('label', 'area')]
[('property', 'http://www.wikidata.org/prop/direct/P281'), ('label', 'postal code')]
[('property', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('property', 'http://www.wikidata.org/prop/direct/P395'), ('label', 'licen

### Get the class of "Florence" (wd:Q2044)

In [7]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q2044 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q1549591'), ('label', 'big city')]
[('class', 'http://www.wikidata.org/entity/Q515'), ('label', 'city')]
[('class', 'http://www.wikidata.org/entity/Q747074'), ('label', 'comune of Italy')]
3


### Get all object properties of "Florence"  (wd:Q2044)

In [8]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q2044 ?property ?object .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1151'), ('label', "topic's main Wikimedia portal")]
[('property', 'http://www.wikidata.org/prop/direct/P131'), ('label', 'located in the administrative territorial entity')]
[('property', 'http://www.wikidata.org/prop/direct/P1313'), ('label', 'office held by head of government')]
[('property', 'http://www.wikidata.org/prop/direct/P1343'), ('label', 'described by source')]
[('property', 'http://www.wikidata.org/prop/direct/P1376'), ('label', 'capital of')]
[('property', 'http://www.wikidata.org/prop/direct/P1441'), ('label', 'present in work')]
[('property', 'http://www.wikidata.org/prop/direct/P1464'), ('label', 'category for people born here')]
[('property', 'http://www.wikidata.org/prop/direct/P1465'), ('label', 'category for people who died here')]
[('property', 'http://www.wikidata.org/prop/direct/P150'), ('label', 'contains administrative territorial entity')]
[('property', 'http://www.wikidata.org/prop/direct/P166'), ('

### Get all the classes connected to "Florence"  (wd:Q2044)

In [9]:
queryString = """
SELECT DISTINCT ?objectClass ?label
WHERE {   
   wd:Q2044 ?property ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q20724701'), ('label', 'city or town in Armenia')]
[('objectClass', 'http://www.wikidata.org/entity/Q56557504'), ('label', 'city of Iran')]
[('objectClass', 'http://www.wikidata.org/entity/Q59542487'), ('label', 'Wikimedia set category')]
[('objectClass', 'http://www.wikidata.org/entity/Q1147395'), ('label', 'district of Turkey')]
[('objectClass', 'http://www.wikidata.org/entity/Q202435'), ('label', 'lieutenancy area of Scotland')]
[('objectClass', 'http://www.wikidata.org/entity/Q13218690'), ('label', 'town in Hungary')]
[('objectClass', 'http://www.wikidata.org/entity/Q8068'), ('label', 'flood')]
[('objectClass', 'http://www.wikidata.org/entity/Q27554677'), ('label', 'former capital')]
[('objectClass', 'http://www.wikidata.org/entity/Q12143'), ('label', 'time zone')]
[('objectClass', 'http://www.wikidata.org/entity/Q21010817'), ('label', 'city of Pennsylvania')]
[('objectClass', 'http://www.wikidata.org/entity/Q56165908'), ('la

### Get all the couples (object property - list of the classes of the objects that are connected to "Florence"  (wd:Q2044) through that object property)

In [10]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?objectClass ; separator = ",")  AS ?objectsClasses) (GROUP_CONCAT(?labelObjectClass ; separator = ",")  AS ?labelsObjectsClasses)
WHERE 
{   
   wd:Q2044 ?property ?object .  
   ?object wdt:P31 ?objectClass .
   
   # Labels
   ?property sc:name ?labelProperty .      
   ?objectClass sc:name ?labelObjectClass .
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P2184'), ('labelProperty', 'history of topic'), ('objectsClasses', 'http://www.wikidata.org/entity/Q28747937'), ('labelsObjectsClasses', 'history of a city')]
[('property', 'http://www.wikidata.org/prop/direct/P6'), ('labelProperty', 'head of government'), ('objectsClasses', 'http://www.wikidata.org/entity/Q5'), ('labelsObjectsClasses', 'human')]
[('property', 'http://www.wikidata.org/prop/direct/P206'), ('labelProperty', 'located in or next to body of water'), ('objectsClasses', 'http://www.wikidata.org/entity/Q573344'), ('labelsObjectsClasses', 'main stream')]
[('property', 'http://www.wikidata.org/prop/direct/P1376'), ('labelProperty', 'capital of'), ('objectsClasses', 'http://www.wikidata.org/entity/Q3024240,http://www.wikidata.org/entity/Q3024240,http://www.wikidata.org/entity/Q3024240,http://www.wikidata.org/entity/Q3024240,http://www.wikidata.org/entity/Q3024240,http://www.wikidata.org/entity/Q3024240,http://www.wikidata

### Get all the couples (object property - list of the  objects that are connected to "Florence"  (wd:Q2044) through that object property)

In [11]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?object ; separator = ",")  AS ?objects) (GROUP_CONCAT(?labelObject; separator = ",")  AS ?labelsObjects)
WHERE 
{   
   wd:Q2044 ?property ?object .  
 
   # Labels
   ?property sc:name ?labelProperty .      
   ?object sc:name ?labelObject
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P2184'), ('labelProperty', 'history of topic'), ('objects', 'http://www.wikidata.org/entity/Q517851'), ('labelsObjects', 'history of Florence')]
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('labelProperty', 'has part'), ('objects', 'http://www.wikidata.org/entity/Q3927266'), ('labelsObjects', 'Historical quarters in Florence')]
[('property', 'http://www.wikidata.org/prop/direct/P6'), ('labelProperty', 'head of government'), ('objects', 'http://www.wikidata.org/entity/Q15976266'), ('labelsObjects', 'Dario Nardella')]
[('property', 'http://www.wikidata.org/prop/direct/P206'), ('labelProperty', 'located in or next to body of water'), ('objects', 'http://www.wikidata.org/entity/Q115457'), ('labelsObjects', 'Arno')]
[('property', 'http://www.wikidata.org/prop/direct/P1376'), ('labelProperty', 'capital of'), ('objects', 'http://www.wikidata.org/entity/Q1273,http://www.wikidata.org/entity/Q16172,http://www.wikidata.org/

### Get the number of poet (wd:Q49757) whose "nationality" is "Italian" (wd:Q38)  that attened university and was born in the same place.

In [6]:
queryString = """
SELECT DISTINCT ?placeOfBirth ?labelPlaceOfBirth (COUNT(DISTINCT ?poet) AS ?numberOfPoets)
WHERE 
{   
    ?poet wdt:P106 wd:Q49757 ;          
          wdt:P27 wd:Q38 ;
          wdt:P69 ?university ;
          wdt:P19 ?placeOfBirth .
          
    ?university wdt:P131 ?placeOfBirth .
    
    #Labels    
    ?placeOfBirth sc:name ?labelPlaceOfBirth .    
}GROUP BY ?placeOfBirth ?labelPlaceOfBirth
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('placeOfBirth', 'http://www.wikidata.org/entity/Q3519'), ('labelPlaceOfBirth', 'Bari'), ('numberOfPoets', '1')]
[('placeOfBirth', 'http://www.wikidata.org/entity/Q1449'), ('labelPlaceOfBirth', 'Genoa'), ('numberOfPoets', '1')]
[('placeOfBirth', 'http://www.wikidata.org/entity/Q2683'), ('labelPlaceOfBirth', 'Parma'), ('numberOfPoets', '2')]
[('placeOfBirth', 'http://www.wikidata.org/entity/Q13629'), ('labelPlaceOfBirth', 'Sassari'), ('numberOfPoets', '1')]
[('placeOfBirth', 'http://www.wikidata.org/entity/Q495'), ('labelPlaceOfBirth', 'Turin'), ('numberOfPoets', '6')]
[('placeOfBirth', 'http://www.wikidata.org/entity/Q13666'), ('labelPlaceOfBirth', 'Messina'), ('numberOfPoets', '1')]
[('placeOfBirth', 'http://www.wikidata.org/entity/Q617'), ('labelPlaceOfBirth', 'Padua'), ('numberOfPoets', '2')]
[('placeOfBirth', 'http://www.wikidata.org/entity/Q326339'), ('labelPlaceOfBirth', 'Üsküdar'), ('numberOfPoets', '1')]
[('placeOfBirth', 'http://www.wikidata.org/entity/Q2759'), ('labe

### Get the number of "poet" (wd:Q49757) whose "nationality" contains "Italy" that attened university and was born in the same place.

In [7]:
queryString = """
SELECT DISTINCT ?placeOfBirth ?labelPlaceOfBirth (COUNT(DISTINCT ?poet) AS ?numberOfPoets)
WHERE 
{   
    ?poet wdt:P106 wd:Q49757 ;
          wdt:P27 ?nationality ;          
          wdt:P69 ?university ;
          wdt:P19 ?placeOfBirth .
    
    ?university wdt:P131 ?placeOfBirth .    
    
    #Labels
    ?nationality sc:name ?labelNationality .
    ?placeOfBirth sc:name ?labelPlaceOfBirth .
    
    FILTER (CONTAINS (?labelNationality, "Italy")) .
}GROUP BY ?placeOfBirth ?labelPlaceOfBirth
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('placeOfBirth', 'http://www.wikidata.org/entity/Q3519'), ('labelPlaceOfBirth', 'Bari'), ('numberOfPoets', '1')]
[('placeOfBirth', 'http://www.wikidata.org/entity/Q1449'), ('labelPlaceOfBirth', 'Genoa'), ('numberOfPoets', '3')]
[('placeOfBirth', 'http://www.wikidata.org/entity/Q2683'), ('labelPlaceOfBirth', 'Parma'), ('numberOfPoets', '2')]
[('placeOfBirth', 'http://www.wikidata.org/entity/Q13629'), ('labelPlaceOfBirth', 'Sassari'), ('numberOfPoets', '1')]
[('placeOfBirth', 'http://www.wikidata.org/entity/Q495'), ('labelPlaceOfBirth', 'Turin'), ('numberOfPoets', '9')]
[('placeOfBirth', 'http://www.wikidata.org/entity/Q13666'), ('labelPlaceOfBirth', 'Messina'), ('numberOfPoets', '1')]
[('placeOfBirth', 'http://www.wikidata.org/entity/Q617'), ('labelPlaceOfBirth', 'Padua'), ('numberOfPoets', '2')]
[('placeOfBirth', 'http://www.wikidata.org/entity/Q326339'), ('labelPlaceOfBirth', 'Üsküdar'), ('numberOfPoets', '1')]
[('placeOfBirth', 'http://www.wikidata.org/entity/Q2759'), ('labe

### Final Query : 2 solutions
For this query there are 2 possible solutions because "italian poets" can be :
- the poets with "nationality" (wtd:P27) equal to "Italy" (wd:Q48)
- the poets with "nationality" (wtd:P27) equal to "Italy" (wd:Q48) or the ones that have "nationality" (wdt:P27) that contains "Italy" in the label or the poets with "birth place" that is in "Italy"
This is due to the fact that, some poets can be italian ones, but they are not born in a before Italy was create, and, because this is not known by the database (snapshot) itself, we can have 2 possible solutions.

Final query for this task

In [11]:
queryString = """
SELECT DISTINCT ?placeOfBirth ?labelPlaceOfBirth (COUNT(DISTINCT ?poet) AS ?numberOfPoets)
WHERE 
{   
    ?poet wdt:P106 wd:Q49757 ;                   
          wdt:P69 ?university ;
          wdt:P19 ?placeOfBirth .
    ?university wdt:P131 ?placeOfBirth . 
        
    {
        ?placeOfBirth wdt:P17 wd:Q48 .    
    }
    UNION
    {
        ?placeOfBirth wdt:P17 ?countryPlaceOfBirth .
        ?countryPlaceOfBirth sc:name ?labelCountryPlaceOfBirth .
        FILTER(CONTAINS(?labelCountryPlaceOfBirth, "Italy")) .
    }
    UNION
    {
        ?poet wdt:P27 wd:Q48 .
    }
    UNION 
    {
        ?poet wdt:P27 ?nationality .
        ?nationality sc:name ?labelNationality .
        FILTER(CONTAINS(?labelNationality, "Italy")) .
    }

    #Labels    
    ?placeOfBirth sc:name ?labelPlaceOfBirth .      

}GROUP BY ?placeOfBirth ?labelPlaceOfBirth
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('placeOfBirth', 'http://www.wikidata.org/entity/Q1903'), ('labelPlaceOfBirth', 'Catania'), ('numberOfPoets', '1')]
[('placeOfBirth', 'http://www.wikidata.org/entity/Q13362'), ('labelPlaceOfBirth', 'Ferrara'), ('numberOfPoets', '1')]
[('placeOfBirth', 'http://www.wikidata.org/entity/Q13629'), ('labelPlaceOfBirth', 'Sassari'), ('numberOfPoets', '1')]
[('placeOfBirth', 'http://www.wikidata.org/entity/Q13138'), ('labelPlaceOfBirth', 'Chieti'), ('numberOfPoets', '1')]
[('placeOfBirth', 'http://www.wikidata.org/entity/Q2634'), ('labelPlaceOfBirth', 'Naples'), ('numberOfPoets', '2')]
[('placeOfBirth', 'http://www.wikidata.org/entity/Q220'), ('labelPlaceOfBirth', 'Rome'), ('numberOfPoets', '16')]
[('placeOfBirth', 'http://www.wikidata.org/entity/Q495'), ('labelPlaceOfBirth', 'Turin'), ('numberOfPoets', '9')]
[('placeOfBirth', 'http://www.wikidata.org/entity/Q2044'), ('labelPlaceOfBirth', 'Florence'), ('numberOfPoets', '9')]
[('placeOfBirth', 'http://www.wikidata.org/entity/Q546'), ('

In [7]:
queryString = """
SELECT DISTINCT ?placeOfBirth ?labelPlaceOfBirth (COUNT(DISTINCT ?poet) AS ?numberOfPoets)
WHERE 
{   
    ?poet wdt:P106 wd:Q49757 ;                   
          wdt:P69 ?university ;
          wdt:P27 wd:Q48 ;
          wdt:P19 ?placeOfBirth .
    ?university wdt:P131 ?placeOfBirth . 

    
    #Labels    
    ?placeOfBirth sc:name ?labelPlaceOfBirth .      

}GROUP BY ?placeOfBirth ?labelPlaceOfBirth
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
Empty


## Task 8
Return the italian poets who won the Nobel Prize in Literature (the result set must be poet IRI and label).

### Get all the list of possible "award received" (wdt:P166) by "poets" (wd:Q49757) and whose label of the award start with "Nob"

In [15]:
queryString = """
SELECT DISTINCT ?award ?labelAward
WHERE 
{   
    ?poet wdt:P106 wd:Q49757 ;
          wdt:P166 ?award .
    ?award sc:name ?labelAward .
    FILTER REGEX (?labelAward ,"^Nob")
}
LIMIT 150
"""
print("Results")
x=run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q37922'), ('labelAward', 'Nobel Prize in Literature')]
[('award', 'http://www.wikidata.org/entity/Q38104'), ('labelAward', 'Nobel Prize in Physics')]
[('award', 'http://www.wikidata.org/entity/Q44585'), ('labelAward', 'Nobel Prize in Chemistry')]
[('award', 'http://www.wikidata.org/entity/Q35637'), ('labelAward', 'Nobel Peace Prize')]
[('award', 'http://www.wikidata.org/entity/Q2192649'), ('labelAward', 'Nobility of the First French Empire')]
[('award', 'http://www.wikidata.org/entity/Q80061'), ('labelAward', 'Nobel Prize in Physiology or Medicine')]
6


### Get all the poet (wd:Q49757) that are connected with "award received" (wdt:P166) to "Nobel Prize in Literature" (wd:Q37922) and that have label for the nationality that contains "Italy"

In [15]:
queryString = """
SELECT DISTINCT ?poet ?labelPoet
WHERE 
{   
    ?poet wdt:P106 wd:Q49757 ;
          wdt:P27 ?nationality ;
          wdt:P166 wd:Q37922 .
    
    #Labels
    ?nationality sc:name ?labelNationality .
    ?poet sc:name ?labelPoet .
    
    FILTER (CONTAINS (?labelNationality, "Italy")) .
}
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q7728'), ('labelPoet', 'Grazia Deledda')]
[('poet', 'http://www.wikidata.org/entity/Q43440'), ('labelPoet', 'Giosuè Carducci')]
[('poet', 'http://www.wikidata.org/entity/Q1403'), ('labelPoet', 'Luigi Pirandello')]
[('poet', 'http://www.wikidata.org/entity/Q83038'), ('labelPoet', 'Salvatore Quasimodo')]
[('poet', 'http://www.wikidata.org/entity/Q83174'), ('labelPoet', 'Eugenio Montale')]
[('poet', 'http://www.wikidata.org/entity/Q765'), ('labelPoet', 'Dario Fo')]
6


### Get all the poet (wd:Q49757) that are connected with "award received" (wdt:P166) to "Nobel Prize in Literature" (wd:Q37922) and that have nationality "Italy" (wd:Q38)

In [17]:
queryString = """
SELECT ?poet ?labelPoet
WHERE 
{   
    ?poet wdt:P106 wd:Q49757 ;
          wdt:P27 wd:Q38 ;
          wdt:P166 wd:Q37922 .
    
    #Labels
    ?poet sc:name ?labelPoet .    
}
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q765'), ('labelPoet', 'Dario Fo')]
[('poet', 'http://www.wikidata.org/entity/Q83038'), ('labelPoet', 'Salvatore Quasimodo')]
[('poet', 'http://www.wikidata.org/entity/Q83174'), ('labelPoet', 'Eugenio Montale')]
3


### Final Query : 2 solutions
For this query there are 2 possible solutions because "italian poets" can be :
- the poets with "nationality" (wtd:P27) equal to "Italy" (wd:Q48)
- the poets with "nationality" (wtd:P27) equal to "Italy" (wd:Q48) or the ones that have "nationality" (wdt:P27) that contains "Italy" in the label or the poets with "birth place" that is in "Italy"
This is due to the fact that, some poets can be italian ones, but they are not born in a before Italy was create, and, because this is not known by the database (snapshot) itself, we can have 2 possible solutions.

Final query for this task

In [13]:
queryString = """
SELECT DISTINCT ?poet ?labelPoet
WHERE 
{   
    ?poet wdt:P106 wd:Q49757 ; 
          wdt:P166 wd:Q37922 ;
          wdt:P19 ?placeOfBirth .
        
    {
        ?placeOfBirth wdt:P17 wd:Q48 .    
    }
    UNION
    {
        ?placeOfBirth wdt:P17 ?countryPlaceOfBirth .
        ?countryPlaceOfBirth sc:name ?labelCountryPlaceOfBirth .
        FILTER(CONTAINS(?labelCountryPlaceOfBirth, "Italy")) .
    }
    UNION
    {
        ?poet wdt:P27 wd:Q48 .
    }
    UNION 
    {
        ?poet wdt:P27 ?nationality .
        ?nationality sc:name ?labelNationality .
        FILTER(CONTAINS(?labelNationality, "Italy")) .
    }
    
    #Labels
    ?poet sc:name ?labelPoet . 
         

}
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q83174'), ('labelPoet', 'Eugenio Montale')]
[('poet', 'http://www.wikidata.org/entity/Q43440'), ('labelPoet', 'Giosuè Carducci')]
[('poet', 'http://www.wikidata.org/entity/Q7728'), ('labelPoet', 'Grazia Deledda')]
[('poet', 'http://www.wikidata.org/entity/Q1403'), ('labelPoet', 'Luigi Pirandello')]
[('poet', 'http://www.wikidata.org/entity/Q83038'), ('labelPoet', 'Salvatore Quasimodo')]
[('poet', 'http://www.wikidata.org/entity/Q765'), ('labelPoet', 'Dario Fo')]
6


In [8]:
queryString = """
SELECT DISTINCT ?poet ?labelPoet
WHERE 
{   
    ?poet wdt:P106 wd:Q49757 ; 
          wdt:P166 wd:Q37922 ;
          wdt:P27 wd:Q48 ;
          wdt:P19 ?placeOfBirth .    
    
    #Labels
    ?poet sc:name ?labelPoet . 
}
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
Empty
